# 04. Automated Screening Pipeline

## AI 기반 저탄소 시멘트 대체재 발견 파이프라인

**목적**: 전체 후보 재료에 대한 자동화 스크리닝 실행

---

### 파이프라인 흐름

```
각 후보 재료에 대해:
1. 구조 생성 (조성 기반)
2. 구조 최적화 (CHGNet)
3. 수화 시스템 생성 (+10 H₂O)
4. MD 시뮬레이션 (10 ps, 300K)
5. 분석 (Ca 용출, Si CN, C-S-H)
6. 점수 계산
```

### 예상 소요 시간
- 후보당: ~15-20분 (GPU)
- 16개 후보: ~4-5시간

## 1. 환경 설정

In [ ]:
import sys
from pathlib import Path
import json
import time
from datetime import datetime

PROJECT_ROOT = Path.cwd().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import src
from src.database import CandidateDatabase
from src.pipeline import ScreeningPipeline, load_config
from src.visualization import plot_screening_comparison, plot_evolution

print(f"Project Root: {PROJECT_ROOT}")
print(f"Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("✓ Modules loaded")

In [ ]:
# GPU 확인
import torch
print(f"\nGPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 설정 및 데이터 로딩

In [ ]:
# 설정 로딩
sim_config = load_config("simulation")
eval_config = load_config("evaluation")

print("Simulation Config:")
print(f"  Optimization Fmax: {sim_config['optimization']['fmax']} eV/Å")
print(f"  MD Duration: {sim_config['md_simulation']['screening']['duration_ps']} ps")
print(f"  Temperature: {sim_config['md_simulation']['screening']['temperature_K']} K")
print(f"  Water Molecules: {sim_config['hydration']['n_water']}")

In [ ]:
# 후보 데이터베이스 로딩
db_path = src.DATA_DIR / "candidates" / "candidates.json"
db = CandidateDatabase(db_path)

print(f"\nCandidate Database: {len(db)} materials")

In [ ]:
# 스크리닝 대상 로딩
screening_list_path = src.DATA_DIR / "candidates" / "screening_list.json"

if screening_list_path.exists():
    with open(screening_list_path, 'r') as f:
        screening_list = json.load(f)
    candidates = [db.get(name) for name in screening_list['candidates'] if db.get(name)]
else:
    candidates = list(db)

print(f"Screening Target: {len(candidates)} candidates")
print("\nCandidate List:")
for i, c in enumerate(candidates, 1):
    print(f"  {i:2}. {c.name:<15} | Tier {c.tier} | CO₂↓ {c.co2_reduction}%")

## 3. 예상 시간 계산

In [ ]:
# 예상 소요 시간
n_candidates = len(candidates)
time_per_candidate = 15  # 분 (GPU 기준)
total_time_min = n_candidates * time_per_candidate

print(f"\n{'='*50}")
print("ESTIMATED TIME")
print(f"{'='*50}")
print(f"  Candidates: {n_candidates}")
print(f"  Time per candidate: ~{time_per_candidate} min")
print(f"  Total estimated: ~{total_time_min} min ({total_time_min/60:.1f} hours)")
print(f"{'='*50}")

## 4. 스크리닝 파이프라인 실행

⚠️ **주의**: 이 셀은 오래 걸립니다 (~4-5시간)

In [ ]:
# 파이프라인 초기화
pipeline = ScreeningPipeline(
    config=sim_config,
    output_dir=src.DATA_DIR / "results",
    verbose=True
)

print("Pipeline initialized")
print(f"Output directory: {pipeline.output_dir}")

In [ ]:
# 스크리닝 실행
print(f"\nStarting screening at {datetime.now().strftime('%H:%M:%S')}")
print("="*60)

start_time = time.time()

results = pipeline.run(candidates)

elapsed = time.time() - start_time
print(f"\nTotal elapsed time: {elapsed/60:.1f} min ({elapsed/3600:.2f} hours)")

## 5. 결과 저장

In [ ]:
# 결과 저장
results_path = pipeline.save_results("pipeline_screening_results.json")
print(f"\n✓ Results saved: {results_path}")

# 에러 확인
if pipeline.errors:
    print(f"\n⚠️ Errors occurred in {len(pipeline.errors)} candidates:")
    for name, error in pipeline.errors.items():
        print(f"  • {name}: {error}")

## 6. 순위 확인

In [ ]:
# 순위 계산
rankings = pipeline.get_rankings()

print("\n" + "="*60)
print("SCREENING RANKINGS")
print("="*60)
print(f"{'Rank':<6} {'Material':<15} {'Score':<8} {'Grade':<6} {'CO₂↓'}")
print("-"*60)

for i, r in enumerate(rankings, 1):
    print(f"{i:<6} {r['name']:<15} {r['score']:<8.1f} {r['grade']:<6} {r['co2_reduction']}%")

print("="*60)

In [ ]:
# 등급별 분류
grades = {'A': [], 'B': [], 'C': [], 'D': []}
for r in rankings:
    grades[r['grade']].append(r['name'])

print("\nGrade Distribution:")
print(f"  A (≥70): {len(grades['A'])} - {', '.join(grades['A']) if grades['A'] else 'None'}")
print(f"  B (≥50): {len(grades['B'])} - {', '.join(grades['B']) if grades['B'] else 'None'}")
print(f"  C (≥30): {len(grades['C'])} - {', '.join(grades['C']) if grades['C'] else 'None'}")
print(f"  D (<30): {len(grades['D'])} - {', '.join(grades['D']) if grades['D'] else 'None'}")

## 7. 상세 결과 확인

In [ ]:
import pandas as pd

# 결과 테이블 생성
table_data = []
for name, result in results.items():
    row = {
        'Material': name,
        'CO2_Reduction': result['co2_reduction'],
        'Ca_Rate': result['analysis']['ca_leaching']['rate_per_ps'],
        'Si_CN': result['analysis']['si_coordination']['mean_cn'],
        'CSH_Pairs': result['analysis']['csh_formation']['max_pairs'],
        'CO2_Score': result['score']['co2_score'],
        'Ca_Score': result['score']['ca_score'],
        'Si_Score': result['score']['si_score'],
        'CSH_Score': result['score']['csh_score'],
        'Total': result['score']['total_score'],
        'Grade': result['score']['grade']
    }
    table_data.append(row)

df = pd.DataFrame(table_data)
df = df.sort_values('Total', ascending=False)

print("\nDetailed Results:")
print(df.to_string(index=False))

In [ ]:
# CSV로 저장
csv_path = src.DATA_DIR / "results" / "pipeline_screening_summary.csv"
df.to_csv(csv_path, index=False)
print(f"\n✓ CSV saved: {csv_path}")

## 8. 시각화

In [ ]:
# 비교 차트
fig1 = plot_screening_comparison(
    results,
    save_path=src.FIGURES_DIR / 'pipeline_screening_comparison.png'
)
plt.show()
print(f"\n✓ Saved: pipeline_screening_comparison.png")

In [ ]:
# 시간 변화 그래프
import matplotlib.pyplot as plt

fig2 = plot_evolution(
    results,
    save_path=src.FIGURES_DIR / 'pipeline_screening_evolution.png'
)
plt.show()
print(f"\n✓ Saved: pipeline_screening_evolution.png")

## 9. Top 후보 선별

In [ ]:
# Top 5 후보
top_n = 5
top_candidates = rankings[:top_n]

print(f"\n{'='*60}")
print(f"TOP {top_n} PROMISING CANDIDATES")
print(f"{'='*60}")

for i, r in enumerate(top_candidates, 1):
    result = results[r['name']]
    print(f"\n{i}. {r['name']}")
    print(f"   Score: {r['score']:.1f}/100 (Grade {r['grade']})")
    print(f"   CO₂ Reduction: {r['co2_reduction']}%")
    print(f"   Ca Leaching: {result['analysis']['ca_leaching']['rate_per_ps']:.3f} Ca/ps")
    print(f"   Si Coordination: {result['analysis']['si_coordination']['mean_cn']:.2f}")
    print(f"   C-S-H Pairs: {result['analysis']['csh_formation']['max_pairs']}")

In [ ]:
# Top 후보 목록 저장
top_list = {
    'timestamp': datetime.now().isoformat(),
    'total_screened': len(results),
    'top_candidates': [r['name'] for r in top_candidates],
    'details': top_candidates
}

top_path = src.DATA_DIR / "results" / "top_candidates.json"
with open(top_path, 'w', encoding='utf-8') as f:
    json.dump(top_list, f, indent=2)

print(f"\n✓ Top candidates saved: {top_path}")

## 10. 요약

In [ ]:
print("\n" + "="*60)
print("SCREENING PIPELINE SUMMARY")
print("="*60)

print(f"""
Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Duration: {elapsed/60:.1f} min ({elapsed/3600:.2f} hours)

Results:
  Total Candidates: {len(candidates)}
  Successful: {len(results)}
  Failed: {len(pipeline.errors)}

Grade Distribution:
  A (≥70): {len(grades['A'])}
  B (≥50): {len(grades['B'])}
  C (≥30): {len(grades['C'])}
  D (<30): {len(grades['D'])}

Top 3 Candidates:
""")

for i, r in enumerate(rankings[:3], 1):
    print(f"  {i}. {r['name']} (Score: {r['score']:.1f}, CO₂↓ {r['co2_reduction']}%)")

print(f"""
Files Generated:
  • {results_path}
  • {csv_path}
  • {top_path}
  • {src.FIGURES_DIR / 'pipeline_screening_comparison.png'}
  • {src.FIGURES_DIR / 'pipeline_screening_evolution.png'}

✅ Screening Pipeline Complete!
""")

print("📋 Next Step: 05_Results_Analysis.ipynb (상세 분석)")

---

## 완료

이 노트북에서 수행한 작업:

1. ✅ 16개 후보 재료 자동 스크리닝
2. ✅ 각 재료별 구조 최적화 + MD 시뮬레이션
3. ✅ Ca 용출, Si 배위수, C-S-H 형성 분석
4. ✅ 점수 계산 및 순위화
5. ✅ Top 후보 선별

**다음 단계**: `05_Results_Analysis.ipynb`에서 결과를 상세 분석하고 논문용 그래프를 생성합니다.